# SRnDeblur_joint 1001 test

## Generator

In [1]:
import argparse
import os
import random
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.backends import cudnn
from torch.autograd import Variable
from torch.utils import data
from torchvision import transforms
from PIL import Image

In [2]:
class Generator(nn.Module):
    def __init__(self, batch_size):
        super(Generator, self).__init__()

        bn = None
        if batch_size == 1:
            bn = False # Instance Normalization
        else:
            bn = True # Batch Normalization

        #============================ upscale ============================#
        self.upscale8 = nn.Sequential(
            # [3x32x32] -> [64x32x32]
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=4, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # [64x32x32] -> [256x32x32]
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            # [256x32x32] -> [64x64x64]
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
            # [64x64x64] -> [256x64x64]
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            # [256x64x64] -> [64x128x128]
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
            # [64x128x128] -> [256x128x128]
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            # [256x128x128] -> [64x256x256]
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),  
            # [64x256x256] -> [3x256x256]
            nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        )
        #============================ upscale ============================#


        # nn.Conv2d(input channel 수, convolution에 의해 생성된 channel 수, kernel size, stride=default 1, padding=default 0)
        # [3x256x256] -> [64x128x128]
        self.conv1 = nn.Conv2d(3, 64, 4, 2, 1)
        # [64x256x256] -> [64x128x128]
#         self.conv1 = nn.Conv2d(64, 64, 3, 1, 1)

        # -> [128x64x64]
        conv2 = [nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1)]
        if bn == True:
            conv2 += [nn.BatchNorm2d(128)]
        else:
            conv2 += [nn.InstanceNorm2d(128)]
        self.conv2 = nn.Sequential(*conv2)

        # -> [256x32x32]
        conv3 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(128, 256, 4, 2, 1)]
        if bn == True:
            conv3 += [nn.BatchNorm2d(256)]
        else:
            conv3 += [nn.InstanceNorm2d(256)]
        self.conv3 = nn.Sequential(*conv3)

        # -> [512x16x16]
        conv4 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(256, 512, 4, 2, 1)]
        if bn == True:
            conv4 += [nn.BatchNorm2d(512)]
        else:
            conv4 += [nn.InstanceNorm2d(512)]
        self.conv4 = nn.Sequential(*conv4)

        # -> [512x8x8]
        conv5 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv5 += [nn.BatchNorm2d(512)]
        else:
            conv5 += [nn.InstanceNorm2d(512)]
        self.conv5 = nn.Sequential(*conv5)

        # -> [512x4x4]
        conv6 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv6 += [nn.BatchNorm2d(512)]
        else:
            conv6 += [nn.InstanceNorm2d(512)]
        self.conv6 = nn.Sequential(*conv6)

        # -> [512x2x2]
        conv7 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv7 += [nn.BatchNorm2d(512)]
        else:
            conv7 += [nn.InstanceNorm2d(512)]
        self.conv7 = nn.Sequential(*conv7)

        # -> [512x1x1]
        conv8 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv8 += [nn.BatchNorm2d(512)]
        else:
            conv8 += [nn.InstanceNorm2d(512)]
        self.conv8 = nn.Sequential(*conv8)

        # -> [512x2x2]
        deconv8 = [nn.ReLU(),
                   nn.ConvTranspose2d(512, 512, 4, 2, 1)]
        if bn == True:
            deconv8 += [nn.BatchNorm2d(512), nn.Dropout(0.5)]
        else:
            deconv8 += [nn.InstanceNorm2d(512), nn.Dropout(0.5)]
        self.deconv8 = nn.Sequential(*deconv8)

        # [(512+512)x2x2] -> [512x4x4]
        deconv7 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 512, 4, 2, 1)]
        if bn == True:
            deconv7 += [nn.BatchNorm2d(512), nn.Dropout(0.5)]
        else:
            deconv7 += [nn.InstanceNorm2d(512), nn.Dropout(0.5)]
        self.deconv7 = nn.Sequential(*deconv7)

        # [(512+512)x4x4] -> [512x8x8]
        deconv6 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 512, 4, 2, 1)]
        if bn == True:
            deconv6 += [nn.BatchNorm2d(512), nn.Dropout(0.5)]
        else:
            deconv6 += [nn.InstanceNorm2d(512), nn.Dropout(0.5)]
        self.deconv6 = nn.Sequential(*deconv6)

        # [(512+512)x8x8] -> [512x16x16]
        deconv5 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 512, 4, 2, 1)]
        if bn == True:
            deconv5 += [nn.BatchNorm2d(512)]
        else:
            deconv5 += [nn.InstanceNorm2d(512)]
        self.deconv5 = nn.Sequential(*deconv5)

        # [(512+512)x16x16] -> [256x32x32]
        deconv4 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 256, 4, 2, 1)]
        if bn == True:
            deconv4 += [nn.BatchNorm2d(256)]
        else:
            deconv4 += [nn.InstanceNorm2d(256)]
        self.deconv4 = nn.Sequential(*deconv4)
        
        # [(512+512)x16x16] -> [256x32x32]
        deconv4_0 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 1, 256, 4, 2, 1)]
        if bn == True:
            deconv4_0 += [nn.BatchNorm2d(256)]
        else:
            deconv4_0 += [nn.InstanceNorm2d(256)]
        self.deconv4_0 = nn.Sequential(*deconv4_0)        

        # [(256+256)x32x32] -> [128x64x64]
        deconv3 = [nn.ReLU(),
                   nn.ConvTranspose2d(256 * 2, 128, 4, 2, 1)]
        if bn == True:
            deconv3 += [nn.BatchNorm2d(128)]
        else:
            deconv3 += [nn.InstanceNorm2d(128)]
        self.deconv3 = nn.Sequential(*deconv3)

        # [(256+256)x32x32] -> [128x64x64]
        deconv3_0 = [nn.ReLU(),
                   nn.ConvTranspose2d(256 * 2, 128, 4, 2, 1)]
        if bn == True:
            deconv3_0 += [nn.BatchNorm2d(128)]
        else:
            deconv3_0 += [nn.InstanceNorm2d(128)]
        self.deconv3_0 = nn.Sequential(*deconv3_0)
        
        # [(128+128)x64x64] -> [64x128x128]
        deconv2 = [nn.ReLU(),
                   nn.ConvTranspose2d(128 * 2, 64, 4, 2, 1)]
        if bn == True:
            deconv2 += [nn.BatchNorm2d(64)]
        else:
            deconv2 += [nn.InstanceNorm2d(64)]
        self.deconv2 = nn.Sequential(*deconv2)
        
        # [(128+128)x64x64] -> [64x128x128]
        deconv2_0 = [nn.ReLU(),
                   nn.ConvTranspose2d(128 * 2, 64, 4, 2, 1)]
        if bn == True:
            deconv2_0 += [nn.BatchNorm2d(64)]
        else:
            deconv2_0 += [nn.InstanceNorm2d(64)]
        self.deconv2_0 = nn.Sequential(*deconv2_0)

        # [(64+64)x128x128] -> [3x256x256]
        self.deconv1 = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 2, 3, 4, 2, 1),
            nn.Tanh()
        )
        
        # [(64+64)x128x128] -> [3x256x256]
        self.deconv1_0 = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 2, 3, 4, 2, 1),
            nn.Tanh()
        )
        
    def forward(self, x):
        
        upx = self.upscale8(x)
        c1 = self.conv1(upx)
        c2 = self.conv2(c1)
        c3 = self.conv3(c2)
        c4 = self.conv4(c3)
        c5 = self.conv5(c4)
        c6 = self.conv6(c5)
        c7 = self.conv7(c6)
        c8 = self.conv8(c7)
        
        d3_0 = self.deconv4_0(c4)
        d3_0 = torch.cat((c3,d3_0), dim=1)
        d2_0 = self.deconv3_0(d3_0)
        d2_0 = torch.cat((c2,d2_0), dim=1)
        d1_0 = self.deconv2_0(d2_0)
        d1_00 = torch.cat((c1,d1_0), dim=1)
        outLR = self.deconv1_0(d1_00)
        
        d7 = self.deconv8(c8)
        d7 = torch.cat((c7, d7), dim=1)
        d6 = self.deconv7(d7)
        d6 = torch.cat((c6, d6), dim=1)
        d5 = self.deconv6(d6)
        d5 = torch.cat((c5, d5), dim=1)
        d4 = self.deconv5(d5)
        d4 = torch.cat((c4, d4), dim=1)
        d3 = self.deconv4(d4)
        d3 = torch.cat((c3, d3), dim=1)
        d2 = self.deconv3(d3)
        d2 = torch.cat((c2, d2), dim=1)
        d1 = self.deconv2(d2)
        d1 = torch.add(d1,d1_0)
        d1 = torch.cat((c1, d1), dim=1)
        outHR = self.deconv1(d1)
#         output = torch.add(outLR,outHR)
#         d1 = torch.cat((c1, d1), dim=1)
#         outHR = self.deconv1(d1)


#         return outLR, outHR
        return upx,outLR, outHR

In [3]:
parser = argparse.ArgumentParser(description='Implementation of Pix2Pix')

# Task
parser.add_argument('--dataroot', required=True, help='path to images (should have subfolders train, val, etc)')
parser.add_argument('--which_direction', type=str, default='AtoB', help='AtoB or BtoA')

# Options
parser.add_argument('--no_resize_or_crop', action='store_true', help='scaling and cropping of images at load time [resize_and_crop|crop|scale_width|scale_width_and_crop]')
parser.add_argument('--no_flip', action='store_true', help='if specified, do not flip the images for data augmentation')
parser.add_argument('--num_epochs', type=int, default=100)
parser.add_argument('--batchSize', type=int, default=1, help='test Batch size')

# misc
parser.add_argument('--model_path', type=str, default='./models')
parser.add_argument('--sample_path', type=str, default='./test_results')
parser.add_argument('--results_txt', type=str, default='./test_MSE_PSNR_SSIM.txt')

##### Helper Functions for Data Loading & Pre-processing
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

In [4]:
##### Helper Functions for Data Loading & Pre-processingclass ImageFolder(data.Dataset):
class ImageFolder(data.Dataset):
    def __init__(self, opt):
        # os.listdir function gives all lists of directory
        self.root = opt.dataroot
        self.no_resize_or_crop = opt.no_resize_or_crop
        self.no_flip = opt.no_flip
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
        self.transformM = transforms.Compose([transforms.ToTensor()])
        #=====================================================================================#
        self.dir_A = os.path.join(opt.dataroot,'celeba_val_1000_LR_8x')
        self.Aimg_paths = list(map(lambda x:os.path.join(self.dir_A,x),os.listdir(self.dir_A)))
        #=====================================================================================#
#         self.dir_AB = os.path.join(opt.dataroot, 'train')
#         self.image_paths = list(map(lambda x: os.path.join(self.dir_AB, x), os.listdir(self.dir_AB)))
        
    def __getitem__(self, index):
        #=====================================================================================#
        # A : 32x32 (blur + LR)
        # B : 256x256 (LR)
        # C : 256x256 (GT)
        # D : 256x256 (fmask)
        A_path = self.Aimg_paths[index]
        trn = A_path.find('LR')
        endn = len(A_path)
        C_path = A_path[:trn]+'HR'+A_path[trn+5:endn-4]+'.jpg'

#         B_path = A_path[:trn]+'GT'+A_path[trn+5:endn-4]+'_mask.jpg'
#         print(A_path, C_path)
        A = Image.open(A_path).convert('RGB')
        C = Image.open(C_path).convert('RGB')
        B = A
        A = C.resize((32,32),Image.BICUBIC)
        C = C.resize((256,256),Image.BICUBIC)
#         B = (C.resize((32,32),Image.BICUBIC)).resize((256,256),Image.BICUBIC)
#         C = C.resize((256,256),Image.BICUBIC)
#         D = D.resize((256,256),Image.BICUBIC)
#         D = D.resize((256,256),Image.BICUBIC)
#         D = torch.zeros(256,256)
#         D = TTF.to_pil_image(D)
        A = self.transform(A)
        B = self.transform(B)
        C = self.transform(C)
#             A = A[:,:32,:32]
        A = A[:,:32,:32]
        B = B[:,:256,:256]
        C = C[:,:256,:256]

        return {'A':A,'B':B, 'C':C,'fname':A_path[trn+6:endn-4]}
        
    def __len__(self):
        return len(self.Aimg_paths)

##### Helper Function for GPU Training
def to_variable(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

##### Helper Function for Math
def denorm(x):
    out = (x+1)/2
    return out.clamp(0,1)

##### Helper Functions for GAN Loss (4D Loss Comparison)
def GAN_Loss(input, target, criterion):
    if target == True:
        tmp_tensor = torch.FloatTensor(input.size()).fill_(1.0)
        labels = Variable(tmp_tensor, requires_grad=False)
    else:
        tmp_tensor = torch.FloatTensor(input.size()).fill_(0.0)
        labels = Variable(tmp_tensor, requires_grad=False)
        
    if torch.cuda.is_available():
        labels = labels.cuda()
        
    return criterion(input, labels)
##### Helper Function for Math
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def to_numpy(x):
    x = x.cpu()
    x = ((x.detach().numpy()+1)/2)
    x = np.transpose(x,(1,2,0))
    return x

def mse(x, y):
    return np.linalg.norm(x - y)


In [5]:
# Pre-settings
cudnn.benchmark = True
global args
args = parser.parse_args(['--dataroot','./comparison_results','--which_direction','AtoB',
                          '--num_epochs','551','--batchSize','4','--no_resize_or_crop',
                          '--model_path','./1031_final_withNevery/models',
                          '--sample_path','./1031_final_withNevery/celebA_results_e551',
                         '--results_txt','./1031_final_withNevery/celebA_PSNRSSIM_e551.txt'])
# 741 751 761 771 781
print(args)

dataset = ImageFolder(args)
data_loader = data.DataLoader(dataset=dataset,
                              batch_size=args.batchSize,
                              shuffle=True,
                              num_workers=2)

if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)
if not os.path.exists(args.sample_path):
    os.makedirs(args.sample_path)

g_path = os.path.join(args.model_path, 'generator-%d.pkl' % (args.num_epochs))
print(g_path)

Namespace(batchSize=4, dataroot='./comparison_results', model_path='./1031_final_withNevery/models', no_flip=False, no_resize_or_crop=True, num_epochs=551, results_txt='./1031_final_withNevery/celebA_PSNRSSIM_e551.txt', sample_path='./1031_final_withNevery/celebA_results_e551', which_direction='AtoB')
./1031_final_withNevery/models/generator-551.pkl


In [6]:
# Load pre-trained model
generator = Generator(args.batchSize)
model_w = torch.load(g_path)
model_w1 = dict()
for k, v in model_w.items():
    nw_name = k[7:]
    model_w1[nw_name] = v
    
generator.load_state_dict(model_w1)
# generator.load_state_dict(torch.load(g_path))
generator.eval()

if torch.cuda.is_available():
    generator = generator.cuda()

total_step = len(data_loader) # For Print Log

from PIL import Image
from skimage import data, img_as_float
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr

f = open(args.results_txt,'w')
mse_in_all, mse_out_all, psnr_in_all, psnr_out_all, ssim_in_all, ssim_out_all = 0,0,0,0,0,0

for i, sample in enumerate(data_loader):

    input_A = sample['A']
    input_A_Bi = sample['B']
    GTHR = sample['C']
    testfileN = sample['fname']

    in_blurLR = to_variable(input_A)
    in_bili = to_variable(input_A_Bi)
    upx, fakeLR, fakeHR = generator(in_blurLR)
    v_GTHR = to_variable(GTHR)
    
    # print the log info
    print('Validation[%d/%d]' % (i + 1, total_step))
    # save the sampled images

    in_Ar = upx[:,0:3,:,:]
    fake_Br = fakeHR[:,0:3,:,:]
    real_Br = v_GTHR[:,0:3,:,:]
    
    for k in range(args.batchSize):
        in_Ar_ = img_as_float(to_numpy(in_Ar[k,:,:,:]))
        fake_Br_ = img_as_float(to_numpy(fake_Br[k,:,:,:]))
        real_Br_ = img_as_float(to_numpy(real_Br[k,:,:,:]))

        mse_in = mse(real_Br_,in_Ar_)
        mse_out = mse(real_Br_,fake_Br_)
        psnr_in = psnr(real_Br_,in_Ar_,data_range=in_Ar_.max()-in_Ar_.min())
        psnr_out = psnr(real_Br_,fake_Br_,data_range=fake_Br_.max()-fake_Br_.min())
        ssim_in = ssim(real_Br_,in_Ar_,data_range=in_Ar_.max()-in_Ar_.min(),multichannel=True)
        ssim_out = ssim(real_Br_,fake_Br_,data_range=fake_Br_.max()-fake_Br_.min(),multichannel=True)

        mse_in_all += mse_in
        mse_out_all += mse_out
        psnr_in_all += psnr_in
        psnr_out_all += psnr_out
        ssim_in_all += ssim_in
        ssim_out_all += ssim_out

        f.write('%s.png \n' % testfileN[k])
        f.write('mse_in : %f, psnr_in : %f, ssim_in : %f \n' % (mse_in, psnr_in, ssim_in))
        f.write('mse_out : %f, psnr_out : %f, ssim_out : %f \n' % (mse_out, psnr_out, ssim_out))
       
        if not os.path.exists(args.sample_path+'/Generated_celebA'):
            os.makedirs(args.sample_path+'/Generated_celebA') 

        torchvision.utils.save_image(denorm(fake_Br[k,:,:,:].data), os.path.join(args.sample_path+'/Generated_celebA', '%s.png' % testfileN[k]))
    
#         res = torch.cat((torch.cat((in_Ar[k,:,:,:], fake_Br[k,:,:,:]), dim=2), real_Br[k,:,:,:]), dim=2)
#         torchvision.utils.save_image(denorm(res.data), os.path.join(args.sample_path, '%s.png' % testfileN[k]))

f.write('Average of MSE PSNR SSIM \n')
f.write('mse_in : %f, psnr_in : %f, ssim_in : %f \n' % (mse_in_all/1200, psnr_in_all/1200, ssim_in_all/1200))
f.write('mse_out : %f, psnr_out : %f, ssim_out : %f \n' % (mse_out_all/1200, psnr_out_all/1200, ssim_out_all/1200))
print('Average of MSE PSNR SSIM \n')
print('mse_in : %f, psnr_in : %f, ssim_in : %f \n' % (mse_in_all/1200, psnr_in_all/1200, ssim_in_all/1200))
print('mse_out : %f, psnr_out : %f, ssim_out : %f \n' % (mse_out_all/1200, psnr_out_all/1200, ssim_out_all/1200))
f.close()

Validation[1/251]
Validation[2/251]
Validation[3/251]
Validation[4/251]
Validation[5/251]
Validation[6/251]
Validation[7/251]
Validation[8/251]
Validation[9/251]
Validation[10/251]
Validation[11/251]
Validation[12/251]
Validation[13/251]
Validation[14/251]
Validation[15/251]
Validation[16/251]
Validation[17/251]
Validation[18/251]
Validation[19/251]
Validation[20/251]
Validation[21/251]
Validation[22/251]
Validation[23/251]
Validation[24/251]
Validation[25/251]
Validation[26/251]
Validation[27/251]
Validation[28/251]
Validation[29/251]
Validation[30/251]
Validation[31/251]
Validation[32/251]
Validation[33/251]
Validation[34/251]
Validation[35/251]
Validation[36/251]
Validation[37/251]
Validation[38/251]
Validation[39/251]
Validation[40/251]
Validation[41/251]
Validation[42/251]
Validation[43/251]
Validation[44/251]
Validation[45/251]
Validation[46/251]
Validation[47/251]
Validation[48/251]
Validation[49/251]
Validation[50/251]
Validation[51/251]
Validation[52/251]
Validation[53/251]
Va

OSError: Traceback (most recent call last):
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-4-ae978b21993e>", line 30, in __getitem__
    A = Image.open(A_path).convert('RGB')
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/site-packages/PIL/Image.py", line 2822, in open
    raise IOError("cannot identify image file %r" % (filename if filename else fp))
OSError: cannot identify image file './comparison_results/celeba_val_1000_LR_8x/.directory'


In [ ]:
args.sample_path+'/Generated_celebA'

In [ ]:
testfileN